In [10]:
import numpy as np
import pandas as pd
import matplotlib
import dicom
from PIL import Image
from PIL import ImageTk
import os
import time
import matplotlib.pyplot as plt
import math
import scipy.ndimage
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg #, NavigationToolbar2TkAgg
from matplotlib.backends.backend_qt4agg import NavigationToolbar2QT
from matplotlib.figure import Figure
import tkinter as tk
from tkinter import ttk
matplotlib.use("TkAgg")
#image_path = 'img_262.jpg'
LARGE_FONT = ("Verdana", 12)
import docx
from keras.models import load_model
import numpy as np
import pickle as pk
import cv2
from docx import Document
global image_path
image_path = "img_262.jpg"

class SeaofBTCapp(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)

        tk.Tk.wm_title(self, "Driver Analyzer")

        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand=True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}

        for F in (IntroPage, UploadPage):
            frame = F(container, self)

            self.frames[F] = frame

            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame(IntroPage)
        
       

    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()


class IntroPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Distracted Driver Detection", font=('Veranda', 30))
        label.pack(pady=10, padx=5)
        button = ttk.Button(self, text="Upload",
                            command=lambda: controller.show_frame(UploadPage))
        button.pack(pady=10, padx=5)


class UploadPage(tk.Frame):
    def __init__(self, parent, controller):
        #global image_path
        tk.Frame.__init__(self, parent)
        label1 = tk.Label(self, text="Loading of Image and Prediction of Class", font = ('Veranda',30))
        label1.pack(pady=10, padx=10)
        label = tk.Label(self, text="Input the image path below:", font=LARGE_FONT)
        label.pack(pady=10, padx=10)
        self.entry = tk.Entry(self)
        self.entry.pack(pady=10, padx=10)

        def printcommand():
            global image_path
            image_path = self.entry.get()
            print("print command")
            print(image_path)
            im = Image.open(image_path)
            im = im.resize((250, 250), Image.ANTIALIAS)
            tkimage = ImageTk.PhotoImage(im)        
            myvar = tk.Label(self, image = tkimage)
            myvar.image = tkimage
            myvar.pack(pady=10, padx=10)
            classes = ["Not distracted", "Texting at Right", "Talking on Phone at Right", "Texting at Left", 
                   "Talking on Phone at Left", "Operating Radio", "Drinking", "Reaching Behind", "Doing Hair/Makeup",
                  "Talking to Passenger"]
            #load the image and predict using the trained model
            input_numpy_image = np.expand_dims(np.transpose(np.array(Image.open(image_path).resize((150,150))), (2,0,1)),0)
            #input_numpy_image = np.expand_dims(np.asarray(cv2.resize(cv2.imread(image_path), (150,150))), axis=0)
            predicted_class = model.predict_classes(input_numpy_image)
            prediction_class = classes[predicted_class[0]]
            
            predicted_class = prediction_class
            
            #display the prediction class
            label = tk.Label(self, text="PREDICTION : ", font=('Veranda', 20))
            label.pack(pady=10, padx=10)
            label.place(x=200, y=600)

            label = tk.Label(self, text=predicted_class, font=('Veranda', 20))
            label.pack(pady=10, padx=5)
            label.place(x=330, y=600)
        
            #display the original class if the image_path exists in train_folder, otherwise print "Not in train data"
            folder_names = image_path.split('/')
            print(folder_names)
            global original_class 
            original_class = "Not in train dataset"
        
            if 'train' in folder_names:
                original_class = classes[int(folder_names[-2][1])]
            label = tk.Label(self, text="ORIGINAL CLASS : ", font=('Veranda', 20))
            label.pack(pady=10, padx=10)
            label.place(x=750, y=600)
        
            label = tk.Label(self, text=original_class, font=('Veranda', 20))
            label.pack(pady=10, padx=5)
            label.place(x=926, y=600)
                
        #load the model whenever UPLOAD is clicked in the HOME page 
        #model = load_model('/Users/Shruthi/Downloads/full_model-40-0.97.hdf5')
        model = load_model('full_model_03-0.94.hdf5')
        #display other buttons in the frame 
        button1 = ttk.Button(self, text="Back",
                             command=lambda: controller.show_frame(IntroPage))
        button1.pack(pady=10, padx=10)


        button3 = ttk.Button(self, text="Analyze Image",
                             command=printcommand)
        button3.pack(pady=10, padx=10)
        
        
        button0 = ttk.Button(self, text="Home",
                             command=lambda: controller.show_frame(IntroPage))
        button0.pack() 
        
        ##button4 = ttk.Button(self, text="Prediction",
                             ##command=lambda: controller.show_frame(PredictPage))
        ##button4.pack(pady=10, padx=10)


'''
class PredictPage(tk.Frame):
    def __init__(self, parent, controller):
        
        
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Prediction", font=('Veranda', 30))
        label.pack(pady=10, padx=10)
        
    #def loading_model(self):    
        classes = ["Not distracted", "Texting at Right", "Talking on Phone at Right", "Texting at Left", 
                   "Talking on Phone at Left", "Operating Radio", "Drinking", "Reaching Behind", "Doing Hair/Makeup",
                  "Talking to Passenger"]
        
        #load the model, image and predict using the trained model
        model = load_model('/Users/Shruthi/Downloads/shruthi_636/training_result/full_model-01-0.11.hdf5')
        #image_path = pk.load(open("/Users/Shruthi/Downloads/path.t","rb"))
        ##doc = Document('path_name.docx')
        ##for para in doc.paragraphs:
            ##path_image.append(para.text)
        global image_path
        print("_________")
        print(image_path)
        input_numpy_image = np.expand_dims(np.asarray(cv2.resize(cv2.imread(image_path), (150,150))), axis=0)
        predicted_class = model.predict_classes(input_numpy_image)
        prediction_class = classes[predicted_class[0]]
        
        #know if the image_path exists in train_folder
        folder_names = image_path.split('/')
        print(folder_names)
        global original_class 
        original_class = "Not in train data"
        
        if 'train' in folder_names:
            original_class = classes[int(folder_names[-2][1])]

        button1 = ttk.Button(self, text="Home",
                             command=lambda: controller.show_frame(IntroPage))
        
        predicted_class = prediction_class

        #val = 'Driver is not distracted'
        label = tk.Label(self, text=predicted_class, font=('Veranda', 20))
        label.pack(pady=10, padx=10)
        
        label = tk.Label(self, text=original_class, font=('Veranda', 20))
        label.pack(pady=10, padx=10)
        
        button1.pack()
'''        

app = SeaofBTCapp()
app.geometry("800x500")
app.mainloop()

print command
/Users/Shruthi/Downloads/state-farm-distracted-driver-detection/imgs/train/c2/img_253.jpg
['', 'Users', 'Shruthi', 'Downloads', 'state-farm-distracted-driver-detection', 'imgs', 'train', 'c2', 'img_253.jpg']
